In [9]:
import os
from typing import List, TypedDict,Union
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI 
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv

load_dotenv("./.env")

class AgentState(TypedDict):
    """ state of the agent."""
    messages: List[Union[HumanMessage, AIMessage]]


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1
)

def process(state: AgentState) -> AgentState:
    """ This node will solve the request you input""" 
    response = llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\n Response Content AI {response.content}")
    print("\n CURRENT STATE" , state["messages"])
    return state


graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()


In [12]:
conversation_history = []
user_input = input("Enter: ")
while user_input != "Exit":
    #add message to history array
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({"messages": conversation_history})
    print(result["messages"])
    conversation_history = result["messages"]
    user_input = input("Enter: ")


with open("loggint.txt", "w") as file:
    file.write("Your conversation log: \n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n")
    
    file.write("End of conversation")

print("Conversation saved to loggint txt\n")


 Response Content AI ¡Hola, David! ¿En qué puedo ayudarte hoy?

 CURRENT STATE [HumanMessage(content='Hol mi nombre es David tengo 40 años', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola, David! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})]
[HumanMessage(content='Hol mi nombre es David tengo 40 años', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola, David! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={})]

 Response Content AI Tu nombre es David. ¿Hay algo más en lo que pueda asistirte?

 CURRENT STATE [HumanMessage(content='Hol mi nombre es David tengo 40 años', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola, David! ¿En qué puedo ayudarte hoy?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cual es mi nombre', additional_kwargs={}, response_metadata={}), AIMessage(content='Tu nombre es David. ¿Hay algo más en lo que pueda asistirte?', additio